In [1]:
import sys
import os

# Add parent directory (../) to Python's search path
sys.path.append(os.path.abspath(".."))

In [2]:
import numpy as np
import pandas as pd


DATA IMPORT

In [3]:
raw_data = pd.read_excel('/workspaces/GEN-AI-DATA-ANALYST/data/e_commerce.xlsx')

In [4]:
raw_data.head(2)

,index,Order ID,Cust ID,Gender,Age,Date,Status,Channel,SKU,Category,Size,Qty,currency,Amount,ship-city,ship-postal-code,ship-country,B2B
0,1,171-1029312-3038738,1029312,Women,44,2022-12-04,Delivered,Myntra,JNE1233-BLUE-KR-031-XXL,kurta,XXL,1,INR,376,MOHALI,140301,IN,False
1,2,405-2183842-2225946,2183842,Women,29,2022-12-04,Delivered,Ajio,SET414-KR-NP-L,Set,L,1,INR,1449,GURUGRAM,122002,IN,False


In [5]:
def scalable_data_profiler(df, sample_size=10000, top_n=5):
    print("📊 BASIC INFO")
    print(f"- Shape: {df.shape}")
    print(f"- Columns: {list(df.columns[:10])}...")  # show only first 10
    print(f"- Memory Usage: ~{df.memory_usage(deep=False).sum() / 1024**2:.2f} MB\n")

    print("🔍 MISSING VALUES (%):")
    missing = df.isnull().mean() * 100
    print(missing[missing > 0].sort_values(ascending=False).head(10), "\n")

    print("🧮 UNIQUE VALUES (Top 10 Columns):")
    unique_counts = df.nunique().sort_values(ascending=False).head(10)
    print(unique_counts, "\n")

    print("📈 NUMERICAL STATS (Sampled if > sample_size):")
    df_sample = df.sample(sample_size) if len(df) > sample_size else df
    print(df_sample.describe(include=[np.number]).T, "\n")

    print("🗂️ CATEGORICAL PREVIEW:")
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    for col in cat_cols[:10]:  # process only first 10 for speed
        print(f"\n🔹 Column: {col}")
        print(f" - Unique: {df[col].nunique()}")
        print(f" - Top {top_n}:\n{df[col].astype(str).value_counts(dropna=False).head(top_n)}")

    print("\n📊 CORRELATION MATRIX (Top Pairs Only):")
    num_cols = df.select_dtypes(include=np.number)
    if num_cols.shape[1] >= 2:
        corr = num_cols.corr().abs().unstack().sort_values(ascending=False)
        corr = corr[corr < 1.0].drop_duplicates().head(10)
        print(corr)
    else:
        print(" - Not enough numerical columns for correlation.")

In [6]:
scalable_data_profiler(raw_data)

📊 BASIC INFO
- Shape: (31047, 18)
- Columns: ['index', 'Order ID', 'Cust ID', 'Gender', 'Age', 'Date', 'Status', 'Channel ', 'SKU', 'Category']...
- Memory Usage: ~4.06 MB

🔍 MISSING VALUES (%):
Series([], dtype: float64) 

🧮 UNIQUE VALUES (Top 10 Columns):
index               31047
Order ID            28471
Cust ID             28437
SKU                  5287
ship-postal-code     4958
ship-city            2603
Amount                769
Age                    61
Date                   36
Size                   11
dtype: int64 

📈 NUMERICAL STATS (Sampled if > sample_size):
                    count          mean           std       min         25%  \
index             10000.0  1.536467e+04  8.969595e+03       1.0     7542.75   
Cust ID           10000.0  4.900196e+06  2.892661e+06    1579.0  2400223.25   
Age               10000.0  3.946170e+01  1.503570e+01      18.0       28.00   
Amount            10000.0  6.826656e+02  2.725935e+02     229.0      481.00   
ship-postal-code  10000.0 

2.0 DATA PREPROCESSING

In [7]:
from genai_dataanalyst.assistant import AnalystAssistant
assistant = AnalystAssistant()

2.1 Solves Categorical Problem

If You standardized the Gender column to ensure it contains only 'Men' or 'Women' by replacing abbreviations: 'W' was changed to 'Women' and 'M' to 'Men'. This makes gender data clean and consistent for analysis.

In [8]:
new_data = assistant.clean(raw_data, prompt="Standardize the 'Gender' column so that it only contains 'Men' or 'Women'. Replace 'W' with 'Women' and 'M' with 'Men'.")

[INFO] [CLEAN] Prompt: Standardize the 'Gender' column so that it only contains 'Men' or 'Women'. Replace 'W' with 'Women' and 'M' with 'Men'.
[DEBUG] [CLEAN] Generated Code:
 df['Gender'] = df['Gender'].replace({'W': 'Women', 'M': 'Men'})


In [9]:
new_data['Gender'].unique() # New data

array(['Women', 'Men'], dtype=object)

In [10]:
raw_data['Gender'].unique()  # Old data

array(['Women', 'Men', 'W', 'M'], dtype=object)

If You cleaned the Qty column by converting text values like 'One' and 'Two' into their numeric forms 1 and 2, so the entire column now contains only numbers. This makes quantity data consistent and ready for calculations

In [11]:
new_data = assistant.clean(new_data, prompt="Standadize all textual quantity values in the Qty column (e.g., 'One', 'Two') to their numeric equivalents (e.g., 1, 2). Ensure the column is entirely numeric after conversion so datatype should be int ")

[INFO] [CLEAN] Prompt: Standadize all textual quantity values in the Qty column (e.g., 'One', 'Two') to their numeric equivalents (e.g., 1, 2). Ensure the column is entirely numeric after conversion so datatype should be int 
[DEBUG] [CLEAN] Generated Code:
 qty_dict = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7, 'Eight': 8, 'Nine': 9, 'Ten': 10}
df['Qty'] = df['Qty'].map(qty_dict).fillna(df['Qty']).astype(int)


<string>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [12]:
new_data['Qty'].unique() # New data


array([1, 2, 4, 3, 5])

In [13]:
raw_data['Qty'].unique()  # Old data

array([1, 'One', 2, 4, 3, 'Two', 5], dtype=object)

In [14]:
new_data.dtypes

index                        int64
Order ID                    object
Cust ID                      int64
Gender                      object
Age                          int64
Date                datetime64[ns]
Status                      object
Channel                     object
SKU                         object
Category                    object
Size                        object
Qty                          int64
currency                    object
Amount                       int64
ship-city                   object
ship-postal-code             int64
ship-country                object
B2B                           bool
dtype: object

3.0 DATA TRANSFORMATION


3.1 Playing With Columns.

I can extract the month and year from the Date column to create two new columns: Order_Month and Order_Year. This makes it easier to analyze sales trends over time, like comparing orders across different months or years.

In [15]:
transformed_data = assistant.clean(new_data, prompt="From the Date column, extract the month and year into two new columns: Order_Month (as full month name) and Order_Year (as 4-digit year).Finally place both the newly created columns after Date column")

[INFO] [CLEAN] Prompt: From the Date column, extract the month and year into two new columns: Order_Month (as full month name) and Order_Year (as 4-digit year).Finally place both the newly created columns after Date column
[DEBUG] [CLEAN] Generated Code:
 df[['Order_Month', 'Order_Year']] = df['Date'].dt.to_period("M").dt.strftime("%B %Y").str.split(" ", expand=True)
df = df[['index', 'Order ID', 'Cust ID', 'Gender', 'Age', 'Date', 'Order_Month', 'Order_Year', 'Status', 'Channel ', 'SKU', 'Category', 'Size', 'Qty', 'currency', 'Amount', 'ship-city', 'ship-postal-code', 'ship-country', 'B2B']]


In [16]:
transformed_data.head(3)

,index,Order ID,Cust ID,Gender,Age,Date,Order_Month,Order_Year,Status,Channel,SKU,Category,Size,Qty,currency,Amount,ship-city,ship-postal-code,ship-country,B2B
0,1,171-1029312-3038738,1029312,Women,44,2022-12-04,December,2022,Delivered,Myntra,JNE1233-BLUE-KR-031-XXL,kurta,XXL,1,INR,376,MOHALI,140301,IN,False
1,2,405-2183842-2225946,2183842,Women,29,2022-12-04,December,2022,Delivered,Ajio,SET414-KR-NP-L,Set,L,1,INR,1449,GURUGRAM,122002,IN,False
2,3,171-1641533-8921966,1641533,Women,67,2022-12-04,December,2022,Delivered,Myntra,SET261-KR-PP-S,Set,S,1,INR,453,KOLKATA,700029,IN,False



Suppose i want to create a new column called Age_Group that classifies each person based on their age: anyone under 20 is labeled a Teenager, those between 20 and 49 are labeled Adult, and those 50 or older are labeled Senior. This helps group customers into age based categories for better analysis.

In [17]:
transformed_data = assistant.clean(transformed_data, prompt="Create a new column of 'Age' column named 'Age_Group'and then place it besides 'Age' column. Based on the Age column: If Age < 20 → Teenager , If Age ≥ 20 and < 50 → Adult , If Age ≥ 50 → Senior")

[INFO] [CLEAN] Prompt: Create a new column of 'Age' column named 'Age_Group'and then place it besides 'Age' column. Based on the Age column: If Age < 20 → Teenager , If Age ≥ 20 and < 50 → Adult , If Age ≥ 50 → Senior
[DEBUG] [CLEAN] Generated Code:
 df['Age_Group'] = pd.cut(df['Age'], bins=[0, 20, 50, np.inf], labels=['Teenager', 'Adult', 'Senior'])
df = df[['index', 'Order ID', 'Cust ID', 'Gender', 'Age', 'Age_Group', 'Date', 'Order_Month', 'Order_Year', 'Status', 'Channel ', 'SKU', 'Category', 'Size', 'Qty', 'currency', 'Amount', 'ship-city', 'ship-postal-code', 'ship-country', 'B2B']]


In [18]:
transformed_data.head(3)

,index,Order ID,Cust ID,Gender,Age,Age_Group,Date,Order_Month,Order_Year,Status,...,SKU,Category,Size,Qty,currency,Amount,ship-city,ship-postal-code,ship-country,B2B
0,1,171-1029312-3038738,1029312,Women,44,Adult,2022-12-04,December,2022,Delivered,...,JNE1233-BLUE-KR-031-XXL,kurta,XXL,1,INR,376,MOHALI,140301,IN,False
1,2,405-2183842-2225946,2183842,Women,29,Adult,2022-12-04,December,2022,Delivered,...,SET414-KR-NP-L,Set,L,1,INR,1449,GURUGRAM,122002,IN,False
2,3,171-1641533-8921966,1641533,Women,67,Senior,2022-12-04,December,2022,Delivered,...,SET261-KR-PP-S,Set,S,1,INR,453,KOLKATA,700029,IN,False


In [19]:
transformed_data = assistant.clean(transformed_data, prompt="Drop the following columns from the dataset currency, ship-country, index")

[INFO] [CLEAN] Prompt: Drop the following columns from the dataset currency, ship-country, index
[DEBUG] [CLEAN] Generated Code:
 df.drop(['currency', 'ship-country', 'index'], axis=1, inplace=True)


In [20]:
transformed_data.head(2)

,Order ID,Cust ID,Gender,Age,Age_Group,Date,Order_Month,Order_Year,Status,Channel,SKU,Category,Size,Qty,Amount,ship-city,ship-postal-code,B2B
0,171-1029312-3038738,1029312,Women,44,Adult,2022-12-04,December,2022,Delivered,Myntra,JNE1233-BLUE-KR-031-XXL,kurta,XXL,1,376,MOHALI,140301,False
1,405-2183842-2225946,2183842,Women,29,Adult,2022-12-04,December,2022,Delivered,Ajio,SET414-KR-NP-L,Set,L,1,1449,GURUGRAM,122002,False


In [21]:
scalable_data_profiler(transformed_data)


📊 BASIC INFO
- Shape: (31047, 18)
- Columns: ['Order ID', 'Cust ID', 'Gender', 'Age', 'Age_Group', 'Date', 'Order_Month', 'Order_Year', 'Status', 'Channel ']...
- Memory Usage: ~3.85 MB

🔍 MISSING VALUES (%):
Series([], dtype: float64) 

🧮 UNIQUE VALUES (Top 10 Columns):
Order ID            28471
Cust ID             28437
SKU                  5287
ship-postal-code     4958
ship-city            2603
Amount                769
Age                    61
Date                   36
Order_Month            12
Size                   11
dtype: int64 

📈 NUMERICAL STATS (Sampled if > sample_size):
                    count          mean           std       min         25%  \
Cust ID           10000.0  4.947294e+06  2.914059e+06    1387.0  2395822.75   
Age               10000.0  3.952740e+01  1.509448e+01      18.0       28.00   
Qty               10000.0  1.006000e+00  9.035937e-02       1.0        1.00   
Amount            10000.0  6.832977e+02  2.701456e+02     229.0      486.00   
ship-postal-

BY USING transform()

In [22]:
transformed_data = assistant.transform(transformed_data, prompt="Encode the 'gender' column using LabelEncoder")

[INFO] [TRANSFORM] Prompt: Encode the 'gender' column using LabelEncoder
[DEBUG] [TRANSFORM] Generated Code:
 import pandas as pd
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])


In [23]:
transformed_data.head(2)
transformed_data['Gender'].unique()

array([1, 0])

ADVANCE DATA TRANSFORMATION SOLVED

1) How can we identify customers who might churn based on inactivity and declining purchase behavior?

RFM-Based Score Threshold
Problem Statement
Use Recency, Frequency, Monetary (RFM) scores to assign a churn risk.
Flag customers as Churn_Risk = True if:

Recency score is low (haven’t bought recently), and

Frequency score is low (rare buyer), and/or

Monetary score is low (not spending much)

In [24]:
transformed_data.head(2)

,Order ID,Cust ID,Gender,Age,Age_Group,Date,Order_Month,Order_Year,Status,Channel,SKU,Category,Size,Qty,Amount,ship-city,ship-postal-code,B2B
0,171-1029312-3038738,1029312,1,44,Adult,2022-12-04,December,2022,Delivered,Myntra,JNE1233-BLUE-KR-031-XXL,kurta,XXL,1,376,MOHALI,140301,False
1,405-2183842-2225946,2183842,1,29,Adult,2022-12-04,December,2022,Delivered,Ajio,SET414-KR-NP-L,Set,L,1,1449,GURUGRAM,122002,False


In [30]:
transformed_data = assistant.transform(transformed_data, prompt="Calculate RFM scores without creating RFM columns for each customer based on columns 'Date', 'Order ID', and 'Amount'. Flag Churn_Risk = True if a customer has low recency, low frequency, and/or low monetary scores (e.g., bottom 25% for each metric).")

[INFO] [TRANSFORM] Prompt: Calculate RFM scores without creating RFM columns for each customer based on columns 'Date', 'Order ID', and 'Amount'. Flag Churn_Risk = True if a customer has low recency, low frequency, and/or low monetary scores (e.g., bottom 25% for each metric).
[DEBUG] [TRANSFORM] Generated Code:
 import pandas as pd
import numpy as np
from datetime import datetime

df['Date'] = pd.to_datetime(df['Date'])
df['Recency'] = (datetime.today() - df['Date']).dt.days
df['Frequency'] = df.groupby('Cust ID')['Order ID'].transform('count')
df['Monetary'] = df.groupby('Cust ID')['Amount'].transform('sum')

recency_quantile = df['Recency'].quantile(0.25)
frequency_quantile = df['Frequency'].quantile(0.25)
monetary_quantile = df['Monetary'].quantile(0.25)

df['Churn_Risk'] = (df['Recency'] < recency_quantile) | (df['Frequency'] < frequency_quantile) | (df['Monetary'] < monetary_quantile)

df.drop(['Recency', 'Frequency', 'Monetary'], axis=1, inplace=True)


In [31]:
transformed_data.head(2)

,Order ID,Cust ID,Gender,Age,Age_Group,Date,Order_Month,Order_Year,Status,Channel,SKU,Category,Size,Qty,Amount,ship-city,ship-postal-code,B2B,Churn_Risk
0,171-1029312-3038738,1029312,1,44,Adult,2022-12-04,December,2022,Delivered,Myntra,JNE1233-BLUE-KR-031-XXL,kurta,XXL,1,376,MOHALI,140301,False,True
1,405-2183842-2225946,2183842,1,29,Adult,2022-12-04,December,2022,Delivered,Ajio,SET414-KR-NP-L,Set,L,1,1449,GURUGRAM,122002,False,True


In [32]:
transformed_data['Churn_Risk'].unique()

array([ True, False])

2 How can we identify repeat customers and understand how frequently they place orders?



Problem Statement:

For each customer (Cust ID) in the dataset:
Determine whether they are a repeat customer (i.e., placed more than one order).
For repeat customers, calculate the average number of days between their purchases using the Date column.

In [34]:
transformed_data = assistant.transform(transformed_data, prompt="Is_Repeat_Customer: mark as Repeat if the customer (Cust ID) has placed more than one order, for single customer Single.")

[INFO] [TRANSFORM] Prompt: Is_Repeat_Customer: mark as Repeat if the customer (Cust ID) has placed more than one order, for single customer Single.


[DEBUG] [TRANSFORM] Generated Code:
 import pandas as pd
import numpy as np

df['Is_Repeat_Customer'] = df.groupby('Cust ID')['Cust ID'].transform(lambda x: 'Repeat' if len(x) > 1 else 'Single')


In [38]:
transformed_data = assistant.transform(transformed_data, prompt="For each customer, Count the total number of orders. Calculate the time difference between their first and last order using the 'Date' column. Derive average order frequency (e.g., orders per month, per quarter, etc.")

[INFO] [TRANSFORM] Prompt: For each customer, Count the total number of orders. Calculate the time difference between their first and last order using the 'Date' column. Derive average order frequency (e.g., orders per month, per quarter, etc.


[DEBUG] [TRANSFORM] Generated Code:
 import pandas as pd
from datetime import timedelta

df['Date'] = pd.to_datetime(df['Date'])

customer_orders = df.groupby('Cust ID')['Order ID'].count().reset_index(name='Total_Orders')

first_last_orders = df.loc[df.groupby('Cust ID')['Date'].idxmin()].merge(df.loc[df.groupby('Cust ID')['Date'].idxmax()], on='Cust ID', suffixes=('_first', '_last'))

first_last_orders['Time_Diff'] = first_last_orders['Date_last'] - first_last_orders['Date_first']

avg_order_freq = first_last_orders['Time_Diff'].apply(lambda x: x.days).divide(first_last_orders['Total_Orders']).reset_index(name='Avg_Order_Freq_Days')

avg_order_freq_month = avg_order_freq['Avg_Order_Freq_Days'].divide(30)
avg_order_freq_quarter = avg_order_freq['Avg_Order_Freq_Days'].divide(90)

result = pd.merge(customer_orders, avg_order_freq, on='Cust ID')
result['Avg_Order_Freq_Month'] = avg_order_freq_month
result['Avg_Order_Freq_Quarter'] = avg_order_freq_quarter
[ERROR] Code execution failed: '

In [ ]:
transformed_data['Is_Repeat_Customer'].unique()

array([False,  True])

In [ ]:
transformed_data = assistant.transform(transformed_data, prompt="create column 'Avg_Purchase_Interval' . For each customer in 'Is_Repeat_Customer' who has True, calculate the average number of days between their orders using the Date column, set False as NaN.")

[INFO] [TRANSFORM] Prompt: create column 'Avg_Purchase_Interval' . For each customer in 'Is_Repeat_Customer' who has True, calculate the average number of days between their orders using the Date column, set False as NaN.
[DEBUG] [TRANSFORM] Generated Code:
 import pandas as pd
import numpy as np

df['Date'] = pd.to_datetime(df['Date'])

df['Avg_Purchase_Interval'] = np.nan

repeat_customers = df[df['Is_Repeat_Customer']]

for customer in repeat_customers['Cust ID'].unique():
    customer_orders = repeat_customers[repeat_customers['Cust ID'] == customer].sort_values('Date')
    purchase_intervals = (customer_orders['Date'] - customer_orders['Date'].shift()).dropna()
    avg_interval = purchase_intervals.mean().days
    df.loc[(df['Cust ID'] == customer) & (df['Is_Repeat_Customer']), 'Avg_Purchase_Interval'] = avg_interval


DATA VISUALIZATION

How does the number of orders vary across different months?

Problem Statement
You want to analyze the monthly trend of orders to understand business seasonality, peak sales periods, or dips in activity. This helps in forecasting, campaign planning, and resource allocation.

Plot a bar chart showing the total number of orders per Order_Month.

In [ ]:
assistant.visualize(transformed_data, prompt="Create a bar chart showing the number of orders per Order_Month. Use the Order_Month column on the x-axis and the count of Order ID on the y-axis.Sort the months chronologically (not alphabetically), and label the axes and title clearly.", name="revenue_bar")

[INFO] [VISUALIZE] Prompt: Create a bar chart showing the number of orders per Order_Month. Use the Order_Month column on the x-axis and the count of Order ID on the y-axis.Sort the months chronologically (not alphabetically), and label the axes and title clearly.


[DEBUG] [VISUALIZE] Code:
 df['Order_Month'] = df['Date'].dt.to_period('M')
fig = go.Figure(data=[go.Bar(x=df.groupby('Order_Month')['Order ID'].count().sort_index().index, y=df.groupby('Order_Month')['Order ID'].count().sort_index().values)])
fig.update_layout(xaxis_title='Order Month', yaxis_title='Number of Orders', title='Orders per Month')
[ERROR] Failed to render chart: Object of type Period is not JSON serializable
[DEBUG] Code was:
 df['Order_Month'] = df['Date'].dt.to_period('M')
fig = go.Figure(data=[go.Bar(x=df.groupby('Order_Month')['Order ID'].count().sort_index().index, y=df.groupby('Order_Month')['Order ID'].count().sort_index().values)])
fig.update_layout(xaxis_title='Order Month', yaxis_title='Number of Orders', title='Orders per Month')


In [ ]:
len(assistant._charts)


1

In [ ]:
for chart in assistant._charts:
    print("Chart Name:", chart["name"])
    print("Prompt Used:", chart["prompt"])
    print("---")


Chart Name: revenue_bar
Prompt Used: Create a bar chart showing the number of orders per Order_Month. Use the Order_Month column on the x-axis and the count of Order ID on the y-axis.Sort the months chronologically (not alphabetically), and label the axes and title clearly.
---


In [ ]:
del assistant._charts[0:1:2]  # Deletes the first, second and third saved chart


In [ ]:
for chart in assistant._charts:
    print("Chart Name:", chart["name"])
    print("Prompt Used:", chart["prompt"])
    print("---")
